In [ ]:
import glob
import numpy as np
import cv2
import os
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from moviepy.editor import VideoFileClip
from IPython.display import HTML

images = glob.glob('camera_cal/calibration*.jpg')
objpoints = []
imgpoints = []

objp = np.zeros((6*9,3), np.float32)
#print (objp)
objp[:,:2] = np.mgrid[0:9,0:6].T.reshape(-1,2)

for fname in images:
    #print (fname)
    img = mpimg.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCorners(gray, (9,6), None)
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)

ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)


In [ ]:
def undistort_image(img, mtx, dist):
    dst = cv2.undistort(img, mtx, dist, None, mtx)
    return dst

In [ ]:
# Output calibration results
out_path = "camera_cal_outputs/"
for fname in images:
    original = mpimg.imread(fname)
    result = undistort_image(original, mtx, dist)
    #print (fname)
    img_name = fname.split('/')[1]
    file_name, file_ext = img_name.split('.')
    out_file_name = file_name + '_out.' + file_ext
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    save_path = os.path.join(out_path, out_file_name)
    plt.imsave(save_path, result)

In [ ]:
# Finding lane lines
def find_lane_pixels(binary_warped):
    #binary_warped = binary_warped[:,:,0]
    histogram = np.sum(binary_warped[binary_warped.shape[0]//2:,:], axis = 0)
    
    
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))
    midpoint = np.int(histogram.shape[0]//2)
    
    quarter_point = np.int(midpoint//2)
    #histogram[midpoint+quarter_point:] = 0
    #plt.plot(histogram)
    leftx_base = np.argmax(histogram[quarter_point:midpoint]) + quarter_point
    rightx_base = np.argmax(histogram[midpoint:midpoint+quarter_point]) + midpoint
    nwindows = 9
    margin = 100
    minpix = 50
    
    window_height = np.int(binary_warped.shape[0]//nwindows)
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    leftx_current = leftx_base
    rightx_current = rightx_base
    left_lane_inds = []
    right_lane_inds = []
    
    for window in range(nwindows):
        win_y_low = binary_warped.shape[0] - (window+1)*window_height
        win_y_high = binary_warped.shape[0] - window*window_height
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        # Draw the windows on the visualization image
        cv2.rectangle(out_img,(win_xleft_low,win_y_low), (win_xleft_high,win_y_high),(0,255,0), 3) 
        cv2.rectangle(out_img,(win_xright_low,win_y_low), (win_xright_high,win_y_high),(0,255,0), 3)
        
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high)
                         & (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high)
                          & (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
        
        if len(good_left_inds) > minpix:
            leftx_current = int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = int(np.mean(nonzerox[good_right_inds]))
            
    left_lane_inds = np.concatenate(left_lane_inds)
    right_lane_inds = np.concatenate(right_lane_inds)
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]

    return leftx, lefty, rightx, righty, out_img

def fit_polynomial(binary_warped):
    leftx, lefty, rightx, righty, out_img = find_lane_pixels(binary_warped)
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    # Generate x and y values for plotting
    ploty = np.linspace(0, binary_warped.shape[0]-1, binary_warped.shape[0] )
    try:
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    except TypeError:
        # Avoids an error if `left` and `right_fit` are still none or incorrect
        print('The function failed to fit a line!')
        left_fitx = 1*ploty**2 + 1*ploty
        right_fitx = 1*ploty**2 + 1*ploty

    ## Visualization ##
    # Colors in the left and right lane regions
    out_img[lefty, leftx] = [255, 0, 0]
    out_img[righty, rightx] = [0, 0, 255]

    # Plots the left and right polynomials on the lane lines
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.imshow(out_img)

    return out_img, left_fit, right_fit, ploty, leftx, rightx, lefty, righty

In [ ]:
# Fit polynomial from previous frame
def fit_poly(img_shape, leftx, lefty, rightx, righty):
    # left_fit: coefficients of left lane
    # right_fit: coefficients of right lane
    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)
    
    ploty = np.linspace(0, img_shape[0] - 1, img_shape[0])
    left_fitx = np.polyval(left_fit, ploty)
    right_fitx = np.polyval(right_fit, ploty)
    
    return left_fitx, right_fitx, ploty, left_fit, right_fit

def search_around_poly(binary_warped, left_fit, right_fit):
    # left_fit: coefficients of left lane
    # right_fit: coefficients of right lane
    margin = 100
    nonzero = binary_warped.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])
    
    leftfit_x = np.polyval(left_fit, nonzeroy)
    rightfit_x = np.polyval(right_fit, nonzeroy)
    left_lane_inds = ((nonzerox > (leftfit_x - margin)) & (nonzerox < (leftfit_x + margin)))
    right_lane_inds = ((nonzerox > (rightfit_x - margin)) & (nonzerox < (rightfit_x + margin)))
    leftx = nonzerox[left_lane_inds]
    lefty = nonzeroy[left_lane_inds]
    rightx = nonzerox[right_lane_inds]
    righty = nonzeroy[right_lane_inds]
    
    left_fitx, right_fitx, ploty, left_fit, right_fit = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    
    # Visualization
    out_img = np.dstack((binary_warped, binary_warped, binary_warped))*255
    window_img = np.zeros_like(out_img)
    out_img[nonzeroy[left_lane_inds], nonzerox[left_lane_inds]] = [255,0,0]
    out_img[nonzeroy[right_lane_inds], nonzerox[right_lane_inds]] = [0,0,255]
    
    left_line_window1 = np.array([np.transpose(np.vstack([left_fitx-margin, ploty]))])
    left_line_window2 = np.array([np.flipud(np.transpose(np.vstack([left_fitx+margin, 
                              ploty])))])
    left_line_pts = np.hstack((left_line_window1, left_line_window2))
    right_line_window1 = np.array([np.transpose(np.vstack([right_fitx-margin, ploty]))])
    right_line_window2 = np.array([np.flipud(np.transpose(np.vstack([right_fitx+margin, 
                              ploty])))])
    right_line_pts = np.hstack((right_line_window1, right_line_window2))    
    
    # Draw the lane onto the warped blank image
    cv2.fillPoly(window_img, np.int_([left_line_pts]), (0,255, 0))
    cv2.fillPoly(window_img, np.int_([right_line_pts]), (0,255, 0))
    result = cv2.addWeighted(out_img, 1, window_img, 0.3, 0)
    
    # Plot the polynomial lines onto the image
    #plt.plot(left_fitx, ploty, color='yellow')
    #plt.plot(right_fitx, ploty, color='yellow')
    #plt.imshow(result)
    ## End visualization steps ##
    
    return result, left_fit, right_fit, ploty

In [ ]:
# Calculate Curvature
def measure_carvature_pixels():
    y_eval = np.max(ploty)
    
    left_curverad = np.sqrt(((2*left_fit[0]*y_eval + left_fit[1])**2 + 1)**3) / np.abs(2*left_fit[0])
    right_curverad = np.sqrt(((2*right_fit[0]*y_eval + right_fit[1])**2 + 1)**3) / np.abs(2*right_fit[0])
    
    return left_curverad, right_curverad

In [ ]:
#def measure_carvature_real(binary_warped, leftx, lefty, rightx, righty, left_fit, right_fit):
def measure_carvature_real(binary_warped, left_fit, right_fit):
    ym_per_pix = 30/720
    xm_per_pix = 3.7/700
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    y_eval = np.max(ploty)
    
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    left_fit_cr = np.polyfit(ploty*ym_per_pix, left_fitx*xm_per_pix, 2)
    right_fit_cr = np.polyfit(ploty*ym_per_pix, right_fitx*xm_per_pix, 2)
    #left_fit_cr = np.polyfit(lefty*ym_per_pix, leftx*xm_per_pix, 2)
    #right_fit_cr = np.polyfit(righty*ym_per_pix, rightx*xm_per_pix, 2)
    
    left_curverad = ((1 + (2*left_fit_cr[0]*y_eval*ym_per_pix + left_fit_cr[1])**2)**1.5) / np.absolute(2*left_fit_cr[0])
    right_curverad = ((1 + (2*right_fit_cr[0]*y_eval*ym_per_pix + right_fit_cr[1])**2)**1.5) / np.absolute(2*right_fit_cr[0])
    
    #car_center = binary_warped.shape[0] / 2
    #lane_center = ((left_fit[0]*binary_warped.shape[0]**2 + left_fit[1]*binary_warped.shape[0] + left_fit[2])
    #              + (right_fit[0]*binary_warped.shape[0]**2 + right_fit[1]*binary_warped.shape[0] + right_fit[2])) / 2
    
    midpoint = (np.mean(left_fitx) + np.mean(right_fitx - left_fitx) // 2)
    midpoint_img = binary_warped.shape[1] // 2
    #print (midpoint)
    #print (midpoint_img)
    #offset = (car_center - lane_center - midpoint) * xm_per_pix
    offset = (midpoint_img - midpoint) * xm_per_pix
    return left_curverad, right_curverad, offset


In [ ]:
# Draw the detected boundaries back to the original image
def draw_detected_area(original, binary_warped, left_fit, right_fit, Minv):
    warp_zeros = np.zeros_like(binary_warped).astype(np.uint8)
    color_warp = np.dstack((warp_zeros, warp_zeros, warp_zeros))

    # Recast the x and y points into usable format for cv2.fillPoly()
    #left_fitx, right_fitx, ploty = fit_poly(binary_warped.shape, leftx, lefty, rightx, righty)
    ploty = np.linspace(0, binary_warped.shape[0] - 1, binary_warped.shape[0])
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    cv2.fillPoly(color_warp, np.int_([pts]), (0,255,0))
    newwarp = cv2.warpPerspective(color_warp, Minv, (binary_warped.shape[1], binary_warped.shape[0]))
    result = cv2.addWeighted(original, 1, newwarp, 0.3, 0)
    
    return result

In [ ]:
# Output carvature and car offset
def print_carvatureAndOffset(img, left_carvature, right_carvature, offset):
    radius = (left_carvature + right_carvature) / 2
    
    offset = np.absolute(offset)
    cv2.putText(img,'Radius:{:04.2f}m.'.format(radius), (100,100), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0,255,0),3, cv2.LINE_AA)
    cv2.putText(img,'Offset:{:04.2f}m.'.format(offset), (100,200), cv2.FONT_HERSHEY_SIMPLEX, 1.8, (0,255,0),3,cv2.LINE_AA)


In [ ]:
# Put everything together to get a pipeline
def region_of_interest(img, vertices):
    """
    Applies an image mask.
    
    Only keeps the region of the image defined by the polygon
    formed from `vertices`. The rest of the image is set to black.
    `vertices` should be a numpy array of integer points.
    """
    #defining a blank mask to start with
    mask = np.zeros_like(img)   
    
    #defining a 3 channel or 1 channel color to fill the mask with depending on the input image
    if len(img.shape) > 2:
        channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
        ignore_mask_color = (255,) * channel_count
    else:
        ignore_mask_color = 255
        
    #filling pixels inside the polygon defined by "vertices" with the fill color    
    cv2.fillPoly(mask, vertices, ignore_mask_color)
    
    #returning the image only where mask pixels are nonzero
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def threshold_bin_image(img):
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS)
    s_channel = hls[:,:,2]
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    lab = cv2.cvtColor(img, cv2.COLOR_RGB2Lab)
    luv = cv2.cvtColor(img, cv2.COLOR_RGB2Luv)
    l_channel = luv[:,:,0]
    b_channel = lab[:,:,2]
    
    sobelx = cv2.Sobel(gray, cv2.CV_64F, 1, 0)
    abs_sobelx = np.absolute(sobelx)
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold x gradient
    thresh_min = 20
    thresh_max = 100
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= thresh_min) & (scaled_sobel <= thresh_max)] = 1
    
    # Threshold S channel
    s_thresh_min = 170
    s_thresh_max = 255
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh_min) & (s_channel <= s_thresh_max)] = 1
    
    # Threshold B channel
    b_thresh_min = 145
    b_thresh_max = 200
    b_binary = np.zeros_like(b_channel)
    b_binary[(b_channel >= b_thresh_min) & (b_channel <= b_thresh_max)] = 1
    
    #Threshold L channel
    l_thresh_min = 215
    l_thresh_max = 255
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh_min) & (l_channel <= l_thresh_max)] = 1
    
    combined_binary = np.zeros_like(sxbinary)
    combined_binary[(sxbinary == 1) | (b_binary == 1) | (l_channel == 1)] = 1
    
    return combined_binary

def perspective_transform_image(img):
    img_size = img.shape[1], img.shape[0]

    src = np.float32([(570,460),
                      (700,460), 
                      (260,680), 
                      (1050,680)])
    dst = np.float32([(450,0),
                      (830,0),
                      (450,img.shape[0]),
                      (830,img.shape[0])])
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size, flags=cv2.INTER_LINEAR)
    
    return warped, M, Minv


In [ ]:
# Line class for video processing
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = np.array([0,0,0], dtype='float')  
        # last n coefficients
        self.recent_fits = []
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #x values for detected line pixels
        self.allx = None  
        #y values for detected line pixels
        self.ally = None
    
    def update(self, line_fit, is_valid, ploty):
        if is_valid == True:
            #if self.best_fit is not None and line_fit is not None:
                #self.diffs = np.absolute(self.best_fit - line_fit)
                #print ("self.diffs")
                #print (self.diffs)
            #if self.diffs[0] > 0.00001 or self.diffs[1] > 1 or self.diffs[2] > 1000:
                #self.detected = False
            #else:
            self.detected = True
            self.recent_xfitted.append(line_fit[0]*ploty**2 + line_fit[1]*ploty + line_fit[2])
            self.recent_fits.append(line_fit)
            if len(self.recent_xfitted) > 4:
                self.recent_xfitted.pop(0)
            if len(self.recent_fits) > 4:
                self.recent_fits.pop(0)
            self.bestx = np.mean(self.recent_xfitted, axis = 0)
            self.current_fit = line_fit
            self.best_fit = np.mean(self.recent_fits, axis = 0) 
        else:
            self.detected = False
            if len(self.recent_xfitted) > 0:
                self.recent_xfitted.pop(0)
            if len(self.recent_fits) > 0:
                self.recent_fits.pop(0)
            if len(self.recent_xfitted) > 0:
                self.bestx = np.mean(self.recent_xfitted, axis = 0)
                self.best_fit = np.mean(self.recent_fits, axis = 0)
                self.current_fit = self.best_fit
                
    def update_curvature(self, curverad, offset):
        curvature_error = 500
        if self.radius_of_curvature is None:
            self.radius_of_curvature = curverad
            return
        else:
            if np.absolute(self.radius_of_curvature - curverad) > curvature_error:
                self.detected = False
            else:
                self.radius_of_curvature = curverad

In [ ]:
# Check if two lanes are parallel
def is_parallel(left_fit, right_fit, ploty):
    if (left_fit is None) or (right_fit is None):
        return False
    '''
    parallel_error = 400 # pixels
    left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
    right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    #print ('parallel error:{}'.format(np.absolute(np.mean(left_fitx - right_fitx))))
    if np.absolute(np.mean(left_fitx - right_fitx)) > parallel_error:
        return False
    '''
    diff = np.absolute(left_fit - right_fit)
    #print ('two fit diff:')
    #print (diff)
    #if diff[0] > 0.00005 or diff[1] > 0.04 or diff[2] > 400:
    if diff[0] > 0.00006 or diff[1] > 0.09 or diff[2] > 600:
        return False
    
    return True

In [ ]:
left_line = Line()
right_line = Line()

def pipeline(img):
    vertices = np.array([[(570,440),(150, img.shape[0]), (1200,img.shape[0]), (750, 450)]], dtype=np.int32)
    undist = undistort_image(img, mtx, dist)
    binary_img = threshold_bin_image(undist)
    binary_img = region_of_interest(binary_img, vertices)
    
    binary_warped, M, Minv = perspective_transform_image(binary_img)
    #plt.imshow(binary_warped)
    # Line processing for video input
    if left_line.detected and right_line.detected:
        result, left_fit, right_fit, ploty = search_around_poly(binary_warped, left_line.best_fit, right_line.best_fit)
    else:
        result, left_fit, right_fit, ploty, leftx, rightx, lefty, righty = fit_polynomial(binary_warped)
        #left_fitx = np.polyval(left_fit, ploty)
        #right_fitx = np.polyval(right_fit, ploty)
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        #plt.show()
    
    is_valid_lanes = True
    if is_parallel(left_fit, right_fit, ploty) == False:
        #print ('not parallel')
        #left_fitx = np.polyval(left_fit, ploty)
        #right_fitx = np.polyval(right_fit, ploty)
        #plt.plot(left_fitx, ploty, color='yellow')
        #plt.plot(right_fitx, ploty, color='yellow')
        #plt.show()
        is_valid_lanes = False
        
    left_line.update(left_fit, is_valid_lanes, ploty)
    right_line.update(right_fit, is_valid_lanes, ploty)
    
    #f, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(24, 9))
    #f.tight_layout()
    #ax1.imshow(binary_img, cmap = 'gray')
    #ax2.imshow(binary_warped, cmap = 'gray')
    #ax3.imshow(result)
    if (left_line.best_fit is not None) and (right_line.best_fit is not None):
        left_curverad, right_curverad, offset = measure_carvature_real(binary_warped, left_line.best_fit, right_line.best_fit)
        left_line.update_curvature(left_curverad, offset)
        right_line.update_curvature(right_curverad, offset)
        img_with_detected_area = draw_detected_area(img, binary_warped, left_line.best_fit, right_line.best_fit, Minv)
        print_carvatureAndOffset(img_with_detected_area, left_curverad, right_curverad, offset)
    else:
        img_with_detected_area = np.copy(img)
        
    return img_with_detected_area


fname = 'test_images/test6.jpg'
img = mpimg.imread(fname)
result = pipeline(img)
#plt.imshow(result)

'''
images = glob.glob('test_images/straight_lines*.jpg')
out_path = "output_images/"

for fname in images:
    original = mpimg.imread(fname)
    result = pipeline(original)
    #print (fname)
    img_name = fname.split('/')[1]
    file_name, file_ext = img_name.split('.')
    out_file_name = file_name + '_polyfit.' + file_ext
    if not os.path.exists(out_path):
        os.makedirs(out_path)
    save_path = os.path.join(out_path, out_file_name)
    plt.imsave(save_path, result)
'''

In [ ]:
# Process video inputs
left_line = Line()
right_line = Line()
video_output = 'project_video_output.mp4'
video_input = VideoFileClip('project_video.mp4')
processed_video = video_input.fl_image(pipeline)
%time processed_video.write_videofile(video_output, audio=False)

In [ ]:
# Challenge video
'''
video_output = 'challenge_video_output.mp4'
video_input = VideoFileClip('challenge_video.mp4').subclip(0,10)
processed_video = video_input.fl_image(pipeline)
%time processed_video.write_videofile(video_output, audio=False)
'''

In [ ]:
# Harder challenge video
'''
video_output = 'harder_challenge_video_output.mp4'
video_input = VideoFileClip('harder_challenge_video.mp4').subclip(0,10)
processed_video = video_input.fl_image(pipeline)
%time processed_video.write_videofile(video_output, audio=False)
'''